Gerekli Kütüphaneler Yüklendi.

In [ ]:

# Kütüphaneler
import pandas as pd
import sqlite3
from datetime import datetime
import os

print("✅ Kütüphaneler yüklendi!")

✅ Kütüphaneler yüklendi!


In [6]:
# Google Drive bağla
from google.colab import drive
drive.mount('/content/drive')

# Veriyi yükle
data_path = '/content/drive/MyDrive/Data-Analyst-Portfolio/Project-1-Supply-Chain/data/processed/supply_chain_clean.csv'
df = pd.read_csv(data_path)

# Tarih kolonlarını düzelt
df['order_date'] = pd.to_datetime(df['order_date'])
df['shipping_date'] = pd.to_datetime(df['shipping_date'])

print(f"✅ Veri yüklendi: {df.shape}")
df.head(3)

Mounted at /content/drive
✅ Veri yüklendi: (180519, 28)


,order_id,order_date,actual_shipping_days,scheduled_shipping_days,delivery_status,customer_country,customer_city,customer_state,order_country,order_region,...,shipping_date,year,month,quarter,day_of_week,month_name,delay_days,is_delayed,on_time,delay_category
0,77202,2018-01-31 22:56:00,3,4,Advance shipping,Puerto Rico,Caguas,PR,Indonesia,Southeast Asia,...,2018-02-03 22:56:00,2018,1,1,2,January,-1,0,0,Zamanında
1,75939,2018-01-13 12:27:00,5,4,Late delivery,Puerto Rico,Caguas,PR,India,South Asia,...,2018-01-18 12:27:00,2018,1,1,5,January,1,1,0,1-3 Gün
2,75938,2018-01-13 12:06:00,4,4,Shipping on time,EE. UU.,San Jose,CA,India,South Asia,...,2018-01-17 12:06:00,2018,1,1,5,January,0,0,1,Zamanında


#**SQL VERİTABANI OLUŞTURMA**

In [ ]:
print("=" * 60)
print("🗄️  SQL VERİTABANI OLUŞTURMA")
print("=" * 60)

# Veritabanı yolu
db_path = '/content/drive/MyDrive/Data-Analyst-Portfolio/Project-1-Supply-Chain/sql/supply_chain.db'

# Klasör yoksa oluştur
os.makedirs(os.path.dirname(db_path), exist_ok=True)

# Veritabanı bağlantısı
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"✅ Veritabanı oluşturuldu: {db_path}")

🗄️  SQL VERİTABANI OLUŞTURMA
✅ Veritabanı oluşturuldu: /content/drive/MyDrive/Data-Analyst-Portfolio/Project-1-Supply-Chain/sql/supply_chain.db


In [ ]:
print("\n" + "=" * 60)
print("📋 TABLO ŞEMASI TASARLAMA")
print("=" * 60)

# TABLO 1: orders (Ana sipariş bilgileri)
schema_orders = """
CREATE TABLE IF NOT EXISTS orders (
    order_id TEXT PRIMARY KEY,
    order_date DATE NOT NULL,
    shipping_date DATE,
    customer_country TEXT,
    customer_city TEXT,
    customer_state TEXT,
    order_country TEXT,
    order_region TEXT,
    shipping_mode TEXT,
    delivery_status TEXT,
    actual_shipping_days INTEGER,
    scheduled_shipping_days INTEGER,
    delay_days REAL,
    is_delayed INTEGER,
    on_time INTEGER,
    late_delivery_risk INTEGER,
    quantity INTEGER,
    sales REAL,
    order_total REAL,
    product_name TEXT,
    category TEXT,
    department TEXT,
    year INTEGER,
    month INTEGER,
    quarter INTEGER,
    day_of_week INTEGER
);
"""

cursor.execute(schema_orders)
print("✅ Tablo şeması oluşturuldu: orders")

# Index'ler (performans için)
indexes = [
    "CREATE INDEX IF NOT EXISTS idx_order_date ON orders(order_date);",
    "CREATE INDEX IF NOT EXISTS idx_country ON orders(customer_country);",
    "CREATE INDEX IF NOT EXISTS idx_region ON orders(order_region);",
    "CREATE INDEX IF NOT EXISTS idx_shipping_mode ON orders(shipping_mode);",
    "CREATE INDEX IF NOT EXISTS idx_category ON orders(category);",
    "CREATE INDEX IF NOT EXISTS idx_is_delayed ON orders(is_delayed);",
]

for idx in indexes:
    cursor.execute(idx)

print("✅ Index'ler oluşturuldu (6 adet)")

conn.commit()


📋 TABLO ŞEMASI TASARLAMA
✅ Tablo şeması oluşturuldu: orders
✅ Index'ler oluşturuldu (6 adet)


Veri Tabloya Aktarıldı.

In [ ]:
print("\n" + "=" * 60)
print("📤 VERİ AKTARIMI")
print("=" * 60)

# Pandas ile direkt SQL'e yaz
df.to_sql('orders', conn, if_exists='replace', index=False)

print(f"✅ {len(df):,} satır 'orders' tablosuna aktarıldı")

# Doğrulama
cursor.execute("SELECT COUNT(*) FROM orders")
count = cursor.fetchone()[0]
print(f"✅ Doğrulama: {count:,} satır veritabanında")

# İlk 3 satırı göster
print("\n📋 İlk 3 Satır (SQL'den):")
sample = pd.read_sql("SELECT * FROM orders LIMIT 3", conn)
print(sample)


📤 VERİ AKTARIMI
✅ 180,519 satır 'orders' tablosuna aktarıldı
✅ Doğrulama: 180,519 satır veritabanında

📋 İlk 3 Satır (SQL'den):
   order_id           order_date  actual_shipping_days  \
0     77202  2018-01-31 22:56:00                     3   
1     75939  2018-01-13 12:27:00                     5   
2     75938  2018-01-13 12:06:00                     4   

   scheduled_shipping_days   delivery_status customer_country customer_city  \
0                        4  Advance shipping      Puerto Rico        Caguas   
1                        4     Late delivery      Puerto Rico        Caguas   
2                        4  Shipping on time          EE. UU.      San Jose   

  customer_state order_country    order_region  ...        shipping_date  \
0             PR     Indonesia  Southeast Asia  ...  2018-02-03 22:56:00   
1             PR         India      South Asia  ...  2018-01-18 12:27:00   
2             CA         India      South Asia  ...  2018-01-17 12:06:00   

   year month  q

EDA'da bulduğumuz sonuçları SQL ile yeniden gözden geçirildi.

In [ ]:
print("\n" + "=" * 80)
print("📊 SQL ANALİZ SORGULARI")
print("=" * 80)

# Tüm sorguları bir dictionary'de topluyoruz
sql_queries = {}


📊 SQL ANALİZ SORGULARI


Genel Performans Metrikleri

In [ ]:
query_1 = """
-- SORGU 1: Genel Performans Özeti
SELECT
    COUNT(*) as total_orders,
    COUNT(DISTINCT customer_country) as unique_countries,
    ROUND(AVG(delay_days), 2) as avg_delay_days,
    ROUND(AVG(is_delayed) * 100, 1) as delayed_pct,
    ROUND(AVG(on_time) * 100, 1) as on_time_pct,
    ROUND(SUM(sales), 2) as total_sales,
    ROUND(AVG(sales), 2) as avg_order_value,
    SUM(quantity) as total_products
FROM orders;
"""

sql_queries['genel_performans'] = query_1

result = pd.read_sql(query_1, conn)
print("\n📊 SORGU 1: Genel Performans Metrikleri")
print("─" * 80)
print(result.to_string(index=False))


📊 SORGU 1: Genel Performans Metrikleri
────────────────────────────────────────────────────────────────────────────────
 total_orders  unique_countries  avg_delay_days  delayed_pct  on_time_pct  total_sales  avg_order_value  total_products
       180519                 2            0.57         57.3         17.8  36784735.01           203.77          384079


Sevkiyat Modu Performansı

In [ ]:
query_2 = """
-- SORGU 2: Sevkiyat Moduna Göre Performans
SELECT
    shipping_mode,
    COUNT(*) as order_count,
    ROUND(AVG(delay_days), 2) as avg_delay,
    ROUND(AVG(is_delayed) * 100, 1) as delayed_pct,
    ROUND(SUM(sales), 2) as total_sales,
    ROUND(AVG(sales), 2) as avg_sales
FROM orders
GROUP BY shipping_mode
ORDER BY avg_delay DESC;
"""

sql_queries['shipping_mode'] = query_2

result = pd.read_sql(query_2, conn)
print("\n📊 SORGU 2: Sevkiyat Modu Performansı")
print("─" * 80)
print(result.to_string(index=False))


📊 SORGU 2: Sevkiyat Modu Performansı
────────────────────────────────────────────────────────────────────────────────
 shipping_mode  order_count  avg_delay  delayed_pct  total_sales  avg_sales
  Second Class        35216       1.99         79.7   7145444.82     202.90
   First Class        27814       1.00        100.0   5674369.76     204.01
      Same Day         9737       0.48         47.8   1942528.56     199.50
Standard Class       107752      -0.00         39.8  22022391.88     204.38


Bölgesel Performans

In [ ]:
query_3 = """
-- SORGU 3: Bölgesel Performans Analizi
SELECT
    order_region,
    COUNT(*) as order_count,
    ROUND(AVG(delay_days), 2) as avg_delay,
    ROUND(AVG(is_delayed) * 100, 1) as delayed_pct,
    ROUND(SUM(sales), 2) as total_sales,
    ROUND(SUM(sales) / SUM(SUM(sales)) OVER () * 100, 1) as sales_share_pct
FROM orders
GROUP BY order_region
ORDER BY avg_delay DESC;
"""

sql_queries['regional'] = query_3

result = pd.read_sql(query_3, conn)
print("\n📊 SORGU 3: Bölgesel Performans")
print("─" * 80)
print(result.to_string(index=False))


📊 SORGU 3: Bölgesel Performans
────────────────────────────────────────────────────────────────────────────────
   order_region  order_count  avg_delay  delayed_pct  total_sales  sales_share_pct
   Central Asia          553       0.65         57.1    109839.93              0.3
 Central Africa         1677       0.64         60.7    327263.02              0.9
     South Asia         7731       0.60         58.5   1553680.92              4.2
 Western Europe        27109       0.60         58.5   5894380.77             16.0
     US Center          5887       0.59         57.1   1151355.77              3.1
    East of USA         6915       0.58         58.0   1371111.99              3.7
 Eastern Europe         3920       0.58         57.4    774266.57              2.1
 South of  USA          4045       0.58         58.1    785783.95              2.1
    East Africa         1852       0.57         57.5    376234.90              1.0
   Eastern Asia         7280       0.57         56.7   14

En Sorunlu 10 Ülke

In [ ]:
query_4 = """
-- SORGU 4: En Çok Gecikme Yaşanan 10 Ülke (Min 100 sipariş)
SELECT
    customer_country,
    COUNT(*) as order_count,
    ROUND(AVG(delay_days), 2) as avg_delay,
    ROUND(AVG(is_delayed) * 100, 1) as delayed_pct,
    ROUND(SUM(sales), 2) as total_sales
FROM orders
GROUP BY customer_country
HAVING order_count >= 100
ORDER BY avg_delay DESC
LIMIT 10;
"""

sql_queries['worst_countries'] = query_4

result = pd.read_sql(query_4, conn)
print("\n📊 SORGU 4: En Sorunlu 10 Ülke (100+ Sipariş)")
print("─" * 80)
print(result.to_string(index=False))


📊 SORGU 4: En Sorunlu 10 Ülke (100+ Sipariş)
────────────────────────────────────────────────────────────────────────────────
customer_country  order_count  avg_delay  delayed_pct  total_sales
         EE. UU.       111146       0.57         57.3  22634493.24
     Puerto Rico        69373       0.56         57.2  14150241.77


Aylık Trend Analizi

In [ ]:
query_5 = """
-- SORGU 5: Aylık Performans Trendi
SELECT
    strftime('%Y-%m', order_date) as year_month,
    COUNT(*) as order_count,
    ROUND(AVG(delay_days), 2) as avg_delay,
    ROUND(AVG(is_delayed) * 100, 1) as delayed_pct,
    ROUND(SUM(sales), 2) as monthly_sales
FROM orders
GROUP BY year_month
ORDER BY year_month;
"""

sql_queries['monthly_trend'] = query_5

result = pd.read_sql(query_5, conn)
print("\n📊 SORGU 5: Aylık Trend (İlk 10 Ay)")
print("─" * 80)
print(result.head(10).to_string(index=False))


📊 SORGU 5: Aylık Trend (İlk 10 Ay)
────────────────────────────────────────────────────────────────────────────────
year_month  order_count  avg_delay  delayed_pct  monthly_sales
   2015-01         5322       0.54         55.9     1051590.08
   2015-02         4729       0.60         57.9      927009.90
   2015-03         5362       0.57         57.2     1051253.69
   2015-04         5126       0.55         56.4     1014463.28
   2015-05         5357       0.56         58.1     1050478.44
   2015-06         5134       0.54         56.3     1024006.17
   2015-07         5299       0.57         57.6     1038081.19
   2015-08         5273       0.60         58.1     1029494.69
   2015-09         5140       0.58         58.9     1018338.60
   2015-10         5302       0.54         57.1     1049154.27


Mevsimsel Pattern

In [ ]:
query_6 = """
-- SORGU 6: Mevsimsel Pattern (Ay Bazlı Ortalama)
SELECT
    month,
    CASE month
        WHEN 1 THEN 'Ocak'
        WHEN 2 THEN 'Şubat'
        WHEN 3 THEN 'Mart'
        WHEN 4 THEN 'Nisan'
        WHEN 5 THEN 'Mayıs'
        WHEN 6 THEN 'Haziran'
        WHEN 7 THEN 'Temmuz'
        WHEN 8 THEN 'Ağustos'
        WHEN 9 THEN 'Eylül'
        WHEN 10 THEN 'Ekim'
        WHEN 11 THEN 'Kasım'
        WHEN 12 THEN 'Aralık'
    END as month_name,
    COUNT(*) as order_count,
    ROUND(AVG(delay_days), 2) as avg_delay,
    ROUND(AVG(is_delayed) * 100, 1) as delayed_pct
FROM orders
GROUP BY month
ORDER BY month;
"""

sql_queries['seasonality'] = query_6

result = pd.read_sql(query_6, conn)
print("\n📊 SORGU 6: Mevsimsel Pattern")
print("─" * 80)
print(result.to_string(index=False))


📊 SORGU 6: Mevsimsel Pattern
────────────────────────────────────────────────────────────────────────────────
 month month_name  order_count  avg_delay  delayed_pct
     1       Ocak        17979       0.57         56.9
     2      Şubat        14529       0.58         57.3
     3       Mart        15919       0.56         57.6
     4      Nisan        15435       0.57         56.7
     5      Mayıs        15976       0.56         57.6
     6    Haziran        15139       0.56         57.1
     7     Temmuz        15922       0.56         57.0
     8    Ağustos        15912       0.58         57.9
     9      Eylül        15489       0.56         57.7
    10       Ekim        12955       0.55         56.9
    11      Kasım        12500       0.57         56.8
    12     Aralık        12764       0.56         57.9


Ürün Kategorisi Analizi

In [ ]:
query_7 = """
-- SORGU 7: En Sorunlu Ürün Kategorileri (Top 10)
SELECT
    category,
    COUNT(*) as order_count,
    ROUND(AVG(delay_days), 2) as avg_delay,
    ROUND(AVG(is_delayed) * 100, 1) as delayed_pct,
    ROUND(SUM(sales), 2) as total_sales
FROM orders
GROUP BY category
ORDER BY avg_delay DESC
LIMIT 10;
"""

sql_queries['categories'] = query_7

result = pd.read_sql(query_7, conn)
print("\n📊 SORGU 7: En Sorunlu 10 Kategori")
print("─" * 80)
print(result.to_string(index=False))


📊 SORGU 7: En Sorunlu 10 Kategori
────────────────────────────────────────────────────────────────────────────────
         category  order_count  avg_delay  delayed_pct  total_sales
Golf Bags & Carts           61       0.77         68.9     10369.39
           Soccer          138       0.72         55.8     26477.05
     Pet Supplies          492       0.71         61.4     41524.80
Strength Training          111       0.67         59.5     54895.53
     Boxing & MMA          423       0.67         59.3     85205.41
         Lacrosse          343       0.66         62.1     39464.79
  As Seen on  TV!           68       0.66         58.8     20597.94
 Women's Clothing          650       0.65         58.6    140283.00
 Tennis & Racquet          328       0.65         57.9     44585.09
         Cameras           592       0.65         62.0    267607.69


 Gecikme Kategorileri Dağılımı

In [ ]:
query_8 = """
-- SORGU 8: Gecikme Kategorileri
SELECT
    CASE
        WHEN delay_days <= 0 THEN 'Zamanında'
        WHEN delay_days <= 3 THEN '1-3 Gün'
        WHEN delay_days <= 7 THEN '4-7 Gün'
        ELSE '7+ Gün'
    END as delay_category,
    COUNT(*) as order_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 1) as percentage
FROM orders
GROUP BY delay_category
ORDER BY
    CASE delay_category
        WHEN 'Zamanında' THEN 1
        WHEN '1-3 Gün' THEN 2
        WHEN '4-7 Gün' THEN 3
        ELSE 4
    END;
"""

sql_queries['delay_categories'] = query_8

result = pd.read_sql(query_8, conn)
print("\n📊 SORGU 8: Gecikme Kategorileri")
print("─" * 80)
print(result.to_string(index=False))


📊 SORGU 8: Gecikme Kategorileri
────────────────────────────────────────────────────────────────────────────────
delay_category  order_count  percentage
     Zamanında        77119        42.7
       1-3 Gün        96417        53.4
       4-7 Gün         6983         3.9


Yüksek Değerli Gecikmeli Siparişler

In [ ]:
query_9 = """
-- SORGU 9: Kritik Gecikmeli Siparişler (Yüksek Değer + Uzun Gecikme)
SELECT
    order_id,
    order_date,
    customer_country,
    shipping_mode,
    delay_days,
    sales,
    category
FROM orders
WHERE is_delayed = 1
  AND sales > (SELECT AVG(sales) FROM orders)
  AND delay_days > 5
ORDER BY sales DESC
LIMIT 20;
"""

sql_queries['critical_delays'] = query_9

result = pd.read_sql(query_9, conn)
print("\n📊 SORGU 9: Kritik Gecikmeli Siparişler (İlk 20)")
print("─" * 80)
print(result.to_string(index=False))


📊 SORGU 9: Kritik Gecikmeli Siparişler (İlk 20)
────────────────────────────────────────────────────────────────────────────────
Empty DataFrame
Columns: [order_id, order_date, customer_country, shipping_mode, delay_days, sales, category]
Index: []


Performans Karşılaştırma (CTE kullanımı)

In [ ]:
query_10 = """
-- SORGU 10: En İyi vs En Kötü Sevkiyat Modu Karşılaştırması (CTE)
WITH mode_performance AS (
    SELECT
        shipping_mode,
        COUNT(*) as order_count,
        ROUND(AVG(delay_days), 2) as avg_delay,
        ROUND(AVG(is_delayed) * 100, 1) as delayed_pct
    FROM orders
    GROUP BY shipping_mode
),
ranked_modes AS (
    SELECT
        *,
        ROW_NUMBER() OVER (ORDER BY avg_delay) as best_rank,
        ROW_NUMBER() OVER (ORDER BY avg_delay DESC) as worst_rank
    FROM mode_performance
)
SELECT
    shipping_mode,
    order_count,
    avg_delay,
    delayed_pct,
    CASE
        WHEN best_rank = 1 THEN 'EN İYİ'
        WHEN worst_rank = 1 THEN 'EN KÖTÜ'
        ELSE 'ORTA'
    END as performance_tier
FROM ranked_modes
WHERE best_rank = 1 OR worst_rank = 1
ORDER BY avg_delay;
"""

sql_queries['mode_comparison'] = query_10

result = pd.read_sql(query_10, conn)
print("\n📊 SORGU 10: En İyi vs En Kötü Mod Karşılaştırması")
print("─" * 80)
print(result.to_string(index=False))


📊 SORGU 10: En İyi vs En Kötü Mod Karşılaştırması
────────────────────────────────────────────────────────────────────────────────
 shipping_mode  order_count  avg_delay  delayed_pct performance_tier
Standard Class       107752      -0.00         39.8           EN İYİ
  Second Class        35216       1.99         79.7          EN KÖTÜ


SQL DOSYALARINI KAYDET

In [ ]:
print("\n" + "=" * 80)
print("💾 SQL DOSYALARINI KAYDETME")
print("=" * 80)

# SQL klasörü
sql_folder = '/content/drive/MyDrive/Data-Analyst-Portfolio/Project-1-Supply-Chain/sql/'

# 1. Şema dosyası
schema_file = sql_folder + 'schema.sql'
with open(schema_file, 'w', encoding='utf-8') as f:
    f.write("-- SUPPLY CHAIN DATABASE SCHEMA\n")
    f.write("-- Created: " + datetime.now().strftime('%Y-%m-%d') + "\n\n")
    f.write(schema_orders)
    f.write("\n\n-- INDEXES\n")
    for idx in indexes:
        f.write(idx + "\n")

print(f"✅ Schema kaydedildi: schema.sql")

# 2. Analiz sorguları dosyası
queries_file = sql_folder + 'analysis_queries.sql'
with open(queries_file, 'w', encoding='utf-8') as f:
    f.write("-- SUPPLY CHAIN ANALYSIS QUERIES\n")
    f.write("-- Created: " + datetime.now().strftime('%Y-%m-%d') + "\n")
    f.write("-- Description: Key analytical queries for supply chain performance\n\n")

    for i, (name, query) in enumerate(sql_queries.items(), 1):
        f.write(f"\n{'='*80}\n")
        f.write(f"-- QUERY {i}: {name.upper()}\n")
        f.write(f"{'='*80}\n")
        f.write(query)
        f.write("\n\n")

print(f"✅ Analiz sorguları kaydedildi: analysis_queries.sql ({len(sql_queries)} sorgu)")

# 3. README dosyası (SQL için)
readme_file = sql_folder + 'README.md'
readme_content = f"""# Supply Chain Database - SQL Analysis

## 📋 Genel Bakış

Bu klasör, Supply Chain projesinin SQL veritabanı ve analiz sorgularını içerir.

- **Veritabanı:** SQLite
- **Tablo Sayısı:** 1 (orders)
- **Satır Sayısı:** {len(df):,}
- **Dönem:** {df['order_date'].min().date()} - {df['order_date'].max().date()}

## 🗄️ Dosyalar

- `supply_chain.db` - SQLite veritabanı dosyası
- `schema.sql` - Tablo şeması ve index'ler
- `analysis_queries.sql` - 10 temel analiz sorgusu

## 📊 Analiz Sorguları

1. **Genel Performans Metrikleri** - Toplam sipariş, satış, gecikme özeti
2. **Sevkiyat Modu Performansı** - Mod bazlı karşılaştırma
3. **Bölgesel Performans** - Bölge bazlı analiz
4. **En Sorunlu Ülkeler** - Kritik coğrafyalar
5. **Aylık Trend** - Zaman serisi analizi
6. **Mevsimsel Pattern** - Ay bazlı ortalamalar
7. **Ürün Kategorisi** - Kategori performansı
8. **Gecikme Kategorileri** - Dağılım analizi
9. **Kritik Siparişler** - Yüksek değer + gecikme
10. **Performans Karşılaştırma** - CTE kullanımı

## 🚀 Kullanım
```sql
-- Veritabanına bağlan
sqlite3 supply_chain.db

-- Sorgu çalıştır
.read analysis_queries.sql
```

## 💡 Teknik Detaylar

- **Normalizasyon:** Tek tablo (denormalized - analiz için optimize)
- **Index'ler:** 6 adet (order_date, country, region, shipping_mode, category, is_delayed)
- **Veri Tipi Optimizasyonu:** INTEGER, REAL, TEXT, DATE
- **Performance:** Index'ler ile hızlı sorgulama

## 📈 Örnek Sorgu
```sql
SELECT
    shipping_mode,
    AVG(delay_days) as avg_delay,
    COUNT(*) as orders
FROM orders
WHERE is_delayed = 1
GROUP BY shipping_mode
ORDER BY avg_delay DESC;
```

---

**Hazırlayan:** [Adınız] | **Tarih:** {datetime.now().strftime('%Y-%m-%d')}
"""

with open(readme_file, 'w', encoding='utf-8') as f:
    f.write(readme_content)

print(f"✅ README oluşturuldu: README.md")

# Bağlantıyı kapat
conn.close()
print("\n✅ Veritabanı bağlantısı kapatıldı")


💾 SQL DOSYALARINI KAYDETME
✅ Schema kaydedildi: schema.sql
✅ Analiz sorguları kaydedildi: analysis_queries.sql (10 sorgu)
✅ README oluşturuldu: README.md

✅ Veritabanı bağlantısı kapatıldı


In [7]:
import pandas as pd

# df = pd.read_csv('/content/drive/MyDrive/Data-Analyst-Portfolio/Project-1-Supply-Chain/data/processed/supply_chain_clean.csv') # df is already loaded in cell bMGiuQn0tOEA

# Ülke sayısı
print("customer_country unique values:")
print(f"Toplam ülke sayısı: {df['customer_country'].nunique()}")
print("\nÜlke listesi:")
print(df['customer_country'].value_counts())
#---

# ## 🎯 NEDEN ÖNEMLİ?

# README'de şu an:
# ```
# 90+ countries
# ```
# yazıyor.

# Eğer gerçekten 2 ülke varsa:
# ```
# 2 countries (Puerto Rico, EE.UU.)
# ```

customer_country unique values:
Toplam ülke sayısı: 2

Ülke listesi:
customer_country
EE. UU.        111146
Puerto Rico     69373
Name: count, dtype: int64


In [8]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Data-Analyst-Portfolio/Project-1-Supply-Chain/data/processed/supply_chain_clean.csv')

# Tüm kolon isimlerini göster
print("KOLONLAR:")
print(df.columns.tolist())

print("\n---")

# order_country varsa
if 'order_country' in df.columns:
    print(f"order_country unique: {df['order_country'].nunique()}")
    print(df['order_country'].value_counts().head(10))

print("\n---")

# order_region
if 'order_region' in df.columns:
    print(f"order_region unique: {df['order_region'].nunique()}")
    print(df['order_region'].value_counts())

KOLONLAR:
['order_id', 'order_date', 'actual_shipping_days', 'scheduled_shipping_days', 'delivery_status', 'customer_country', 'customer_city', 'customer_state', 'order_country', 'order_region', 'product_name', 'category', 'department', 'quantity', 'sales', 'order_total', 'shipping_mode', 'late_delivery_risk', 'shipping_date', 'year', 'month', 'quarter', 'day_of_week', 'month_name', 'delay_days', 'is_delayed', 'on_time', 'delay_category']

---
order_country unique: 164
order_country
Estados Unidos    24840
Francia           13222
México            13172
Alemania           9564
Australia          8497
Brasil             7987
Reino Unido        7302
China              5758
Italia             4989
India              4783
Name: count, dtype: int64

---
order_region unique: 23
order_region
Central America    28341
Western Europe     27109
South America      14935
Oceania            10148
Northern Europe     9792
Southeast Asia      9539
Southern Europe     9431
Caribbean           8318
West